In [1]:
import pandas as pd, numpy as np, datetime as dt
from MyModules.features import new_datetime_complete
from MyModules.candlePlotting import plot_ticks
import http, json

def get_oanda_candles(instr, period, start):
    conn = http.client.HTTPSConnection("api-fxpractice.oanda.com")
    headers = {"Authorization": "Bearer ***REMOVED***"}
    url = "https://api-fxpractice.oanda.com/v3/instruments/{}/candles?&from={}&granularity={}&alignmentTimezone=America/Los_Angeles"\
            .format(instr, start, period)
    conn.request("GET", url, None, headers)
    resp = conn.getresponse()
    if resp.status in (201, 200):
        return json.loads(resp.read())
    else:
        raise http.client.HTTPException("Error in HTTP request (status: " + str(resp.status) + "):\n" + str(resp.read()))

def get_df_deltas(instr, period):
## This function assumes that delta are from after the 2nd-to-last candle, since last hasn't closed yet
    df = pd.read_csv(r'Datasets/{} {}.csv'.format(instr, period), parse_dates=[0], index_col=0)
    #df = df[(df.Open != df.High) & (df.Open != df.Low) & (df.Open != df.Close)]  # this doesn't work as intended
# Get delta candles from last date of df
    delta_candles = None
    while delta_candles is None:
        try:
            delta_candles = get_oanda_candles(instr, period, df.index[-1].strftime('%Y-%m-%dT%H:%M:%S.000000000Z'))
        except:
            pass
    delta_candles = pd.DataFrame(delta_candles['candles']).drop(['complete', 'volume'], axis=1)
    delta_candles['time'] = pd.to_datetime(delta_candles['time'])
    delta_candles = delta_candles.set_index('time')

    cols = ('Open', 'High', 'Low', 'Close')
    for c in cols:
        delta_candles[c] = delta_candles['mid'].apply(lambda r: float(r['{}'.format(c[0].lower())]))
    delta_candles = delta_candles.drop('mid', axis=1)

    if df.index[-1] == delta_candles.index[0]: df = df.drop(df.index[-1])  # Drop last point of df assuming it's the same date as the first delta candle

    delta_candles['Rejection'] = np.nan
    df = df.append(delta_candles)
    
    return df

def split_df(instr, period, len_longterm, len_window):
    df = get_df_deltas(instr, period)

    min_w_lt = max(0, len(df)-(len_longterm+1))
    min_w = max(0, len(df)-(len_window+1))
    max_w = len(df)-2                          # execute new_datetime on penultimate candle for analysis
                                               # to wait til close of the candle to make proper IPDE's
    df['Rejection'].iloc[:min_w+1] = np.nan # remove Rejection price on points leading up to df_window
    df_longterm = df.iloc[min_w_lt:max_w, :].copy()
    df_window = df.iloc[min_w:max_w, :].copy()
    df_lastclosed = df.iloc[max_w].copy()
    df_last = df.iloc[-1].copy()
    
    df_window = df_window_cols(df_window)

    return df_longterm, df_window, df_lastclosed, df_last

def df_window_cols(df_window):
    cols = ['Volume', 'Candle Pattern', 'Same-sized Candle Trend Rejection', 'Engulfing Pattern', 'Immediate Trend Direction', 'Rejection',
            'Near Short-term Control', 'Near Long-term Control', 'In Excess Above Short-term Value', 'In Excess Below Short-term Value',
            'In Excess Above Long-term Value', 'In Excess Below Long-term Value', 'Rejected Short-term Control', 'Rejected Long-term Control',
            'Rejected Short-term Upper Limit', 'Rejected Short-term Lower Limit', 'Rejected Long-term Upper Limit', 'Rejected Long-term Lower Limit',
            'Near Short-term SR', 'Near Long-term SR', 'Near Sloped SR', 'Rejected Short-term SR line', 'Rejected Long-term SR line',
            'Rejected Sloped SR line', 'Long-term Trend Direction', 'In Excess of Long-term Value Area, Trend-following',
            'In Excess of Long-term Value Area, Counter-trend', 'Rejected Fibo level 236', 'Rejected Fibo level 382', 'Rejected Fibo level 618',
            'Closed above previous (green)', 'Close below previous (red)']
    for c in cols:
        if c != 'Rejection': df_window[c] = np.nan
    df_window = df_window[['Open', 'High', 'Low', 'Close'] + cols]
    
    return df_window

/home/mt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/mt/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:304: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


### Default ratios

In [2]:
cur = 'EUR_USD'
time = 'D'

print(cur, time)
df_longterm, df_window, df_lastclosed, df_last = split_df(cur, time, len_longterm=550, len_window=120)

# FIND SUITABLE TOL
# same_sized_candle_trend_rejection - 1.25
# avg_sr_lines - 0.0020  long term
#                0.0015  short term
# get_sloped_SRlines - 0.00175
# new_datetime_complete - 0.0008

EUR_USD D


In [4]:
maxi = max(df_window.Close) if max(df_window.Close) > max(df_window.Open) else max(df_window.Open)
mini = min(df_window.Close) if min(df_window.Close) < min(df_window.Open) else min(df_window.Open)
print(maxi, mini)

1.19525 1.13414


In [5]:
maxi - mini

0.06111

### Ratio for Monthly

In [44]:
print('same_sized_candle_trend_rejection ', 1.25 * 7.4318442153493685, sep='\n', end='\n\n')
print('avg_sr_lines long term ', 0.0020 * 7.4318442153493685, sep='\n', end='\n\n')
print('avg_sr_lines short term ', 0.0015 * 7.4318442153493685, sep='\n', end='\n\n')
print('get_sloped_SRlines ', 0.00175 * 7.4318442153493685, sep='\n', end='\n\n')
print('new_datetime_complete ', 0.0008 * 7.4318442153493685, sep='\n', end='\n\n')

same_sized_candle_trend_rejection 
9.289805269186711

avg_sr_lines long term 
0.014863688430698737

avg_sr_lines short term 
0.011147766323024054

get_sloped_SRlines 
0.013005727376861394

new_datetime_complete 
0.005945475372279495



### Ratio for Weekly

In [45]:
print('same_sized_candle_trend_rejection ', 1.25 * r, sep='\n', end='\n\n')
print('avg_sr_lines long term ', 0.0020 * r, sep='\n', end='\n\n')
print('avg_sr_lines short term ', 0.0015 * r, sep='\n', end='\n\n')
print('get_sloped_SRlines ', 0.00175 * r, sep='\n', end='\n\n')
print('new_datetime_complete ', 0.0008 * r, sep='\n', end='\n\n')

same_sized_candle_trend_rejection 
4.1228931435117

avg_sr_lines long term 
0.0065966290296187195

avg_sr_lines short term 
0.004947471772214039

get_sloped_SRlines 
0.005772050400916379

new_datetime_complete 
0.0026386516118474877



### Ratio for 4-Hourly:  _Keep the same as such as the Daily's_

In [ ]:
cur = 'EUR_USD'
time = 'H4'

print(cur, time)
df_longterm, df_window, df_lastclosed, df_last = split_df(cur, time, len_longterm=550, len_window=120)

In [ ]:
maxi = max(df_window.Close) if max(df_window.Close) > max(df_window.Open) else max(df_window.Open)
mini = min(df_window.Close) if min(df_window.Close) < min(df_window.Open) else min(df_window.Open)
print(maxi, mini)

In [ ]:
print(maxi - mini)
r = (maxi - mini) / 0.06111
print(r)

In [ ]:
print('same_sized_candle_trend_rejection ', 1.25 * r, sep='\n', end='\n\n')
print('avg_sr_lines long term ', 0.0020 * r, sep='\n', end='\n\n')
print('avg_sr_lines short term ', 0.0015 * r, sep='\n', end='\n\n')
print('get_sloped_SRlines ', 0.00175 * r, sep='\n', end='\n\n')
print('new_datetime_complete ', 0.0008 * r, sep='\n', end='\n\n')